In [3]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tslearn.metrics import cdist_dtw
from tslearn.clustering import TimeSeriesKMeans
from sklearn.metrics import silhouette_score

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [10]:
data_path = '/Users/rheajaisinghani/Desktop/Project DS/Dexcom Data/'

csv_files = glob.glob(data_path + '*.csv')

df_list = []

for file in csv_files:
    df = pd.read_csv(file, skiprows=range(1, 11))
    df_list.append(df)

dexcom_df = pd.concat(df_list, ignore_index=True)

dexcom_df.shape

(339429, 14)

In [11]:
dexcom_df.columns

Index(['Index', 'Timestamp (YYYY-MM-DDThh:mm:ss)', 'Event Type',
       'Event Subtype', 'Patient Info', 'Device Info', 'Source Device ID',
       'Glucose Value (mg/dL)', 'Insulin Value (u)', 'Carb Value (grams)',
       'Duration (hh:mm:ss)', 'Glucose Rate of Change (mg/dL/min)',
       'Transmitter Time (Long Integer)', 'Transmitter ID'],
      dtype='object')

In [12]:
timeseries_data = dexcom_df[['Glucose Value (mg/dL)']]

In [13]:
dexcom_df.loc[dexcom_df['Glucose Value (mg/dL)'] == 'Low', 'Glucose Value (mg/dL)'] = 39
dexcom_df.loc[dexcom_df['Glucose Value (mg/dL)'] == 'High', 'Glucose Value (mg/dL)'] = 401

In [14]:
dexcom_df['Timestamp'] = pd.to_datetime(dexcom_df['Timestamp (YYYY-MM-DDThh:mm:ss)'])
dexcom_df.head()

,Index,Timestamp (YYYY-MM-DDThh:mm:ss),Event Type,Event Subtype,Patient Info,Device Info,Source Device ID,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams),Duration (hh:mm:ss),Glucose Rate of Change (mg/dL/min),Transmitter Time (Long Integer),Transmitter ID,Timestamp
0,11,2022-09-15T00:01:06,EGV,NaN,NaN,NaN,iOS G6,126,NaN,NaN,NaN,NaN,7024275.0,8XSKAL,2022-09-15 00:01:06
1,12,2022-09-15T00:06:05,EGV,NaN,NaN,NaN,iOS G6,128,NaN,NaN,NaN,NaN,7024575.0,8XSKAL,2022-09-15 00:06:05
2,13,2022-09-15T00:11:05,EGV,NaN,NaN,NaN,iOS G6,128,NaN,NaN,NaN,NaN,7024875.0,8XSKAL,2022-09-15 00:11:05
3,14,2022-09-15T00:16:05,EGV,NaN,NaN,NaN,iOS G6,130,NaN,NaN,NaN,NaN,7025175.0,8XSKAL,2022-09-15 00:16:05
4,15,2022-09-15T00:21:06,EGV,NaN,NaN,NaN,iOS G6,132,NaN,NaN,NaN,NaN,7025475.0,8XSKAL,2022-09-15 00:21:06


In [15]:
# Group by week using pd.Grouper (by default, groups by end-of-week, usually Sunday)
weekly_groups = dexcom_df.groupby(pd.Grouper(key='Timestamp', freq='W'))

# Create a list of weekly time series for the glucose values.
# Each element in weekly_series is a numpy array representing one week's data.
weekly_series = [group['Glucose Value (mg/dL)'].values
                 for _, group in weekly_groups if not group.empty]

In [18]:
max_length = max(len(series) for series in weekly_series)

padded_series = []
for series in weekly_series:
    padded = np.pad(series, (0, max_length - len(series)), mode='edge')
    padded_series.append(padded)

data_array = np.array(padded_series)
data_array = data_array[..., np.newaxis]  # Add a feature dimension

print("Shape of data_array:", data_array.shape)

Shape of data_array: (172, 4008, 1)


In [20]:
normalized_data = []
scaler = MinMaxScaler()
for series in data_array:
    normalized_series = scaler.fit_transform(series)
    normalized_data.append(normalized_series)
normalized_data = np.array(normalized_data)

sample_size = 50  # adjust based on your needs
sampled_data = normalized_data[:sample_size]

In [21]:
distance_matrix = cdist_dtw(sampled_data, n_jobs=-1)

In [24]:
silhouette_scores = []
n_clusters_range = range(2, 6)  # try 2 to 5 clusters

for n_clusters in n_clusters_range:
    kmeans = TimeSeriesKMeans(n_clusters=n_clusters, metric="dtw", random_state=0)
    print('kmeans')
    clusters = kmeans.fit_predict(sampled_data)
    print('clusters')
    score = silhouette_score(distance_matrix, clusters, metric="precomputed")
    print('score')
    silhouette_scores.append(score)
    print('silhouette_scores')
    print(f'Clusters: {n_clusters}, Silhouette Score: {score}')

# Plot silhouette scores
plt.plot(list(n_clusters_range), silhouette_scores, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')
plt.show()

kmeans


KeyboardInterrupt: 